In [1]:
# Install necessary libraries
!pip install duckdb lightgbm optuna category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 67.4 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 68.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 82.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.2/582.2 kB 32.3 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import gc
del test_path
gc.collect()

1540

In [2]:
# ===========================
# Load and Sample Training Data (DuckDB)
# ===========================
# This section loads the AeroClub RecSys Cup 2025 training dataset using DuckDB for efficient querying.
# 
# 🔹 We filter sessions (`ranker_id`) that have at least 10 flight options to ensure meaningful ranking.
# 🔹 From these, we randomly sample 10,000 sessions for faster experimentation (adjust as needed for memory).
# 🔹 The query joins back to get full records for the sampled sessions.
# 🔹 DuckDB is preferred here for fast, SQL-like filtering on large Parquet files without loading everything into memory.
#
# 👉 You can disable sampling for full dataset access by switching to the fallback query below.
# # (Optional) Load entire dataset without filtering or sampling
# query = f"""
#     SELECT *
#     FROM parquet_scan('{train_path}')
# """


import duckdb
import pandas as pd

train_path = "/kaggle/input/aeroclub-recsys-cup-2025-1/train.parquet"
test_path = "/kaggle/input/aeroclub-recsys-cup-2025-1/test.parquet"

con = duckdb.connect()

# Correct session sampling with minimum flight options
query = f"""
    WITH valid_sessions AS (
        SELECT ranker_id
        FROM parquet_scan('{train_path}')
        GROUP BY ranker_id
        HAVING COUNT(*) >= 10  -- Sessions with at least 10 flight options
    ),
    sampled_sessions AS (
        SELECT ranker_id
        FROM valid_sessions
        USING SAMPLE 10000  -- Sample 5000 sessions (adjust based on memory)
    )
    SELECT t.*
    FROM parquet_scan('{train_path}') t
    JOIN sampled_sessions s ON t.ranker_id = s.ranker_id
"""

# query = f"""
#     SELECT *
#     FROM parquet_scan('{train_path}')
# """

# Execute query and convert to DataFrame
df = con.query(query).to_df()
con.close()

# Check session stats
session_counts = df['ranker_id'].value_counts()
print(f"Loaded {len(df)} rows from {len(session_counts)} sessions")
print(f"Min options per session: {session_counts.min()}")
print(f"Max options per session: {session_counts.max()}")
print(f"Selected flights: {df['selected'].sum()} ({(df['selected'].mean()*100):.2f}%)")

Loaded 2007739 rows from 10000 sessions
Min options per session: 10
Max options per session: 7841
Selected flights: 10000 (0.50%)


In [3]:
# ===========================
# Pandas Display Settings
# ===========================
# These options improve the readability of large DataFrames during development.
#
# 🔹 display.max_rows:    Show up to 500 rows (instead of truncating after 10).
# 🔹 display.max_columns: Show up to 500 columns (helps when working with wide datasets).
# 🔹 display.width:       Sets the display width to 1000 characters so rows don’t wrap across lines.
#
# 👉 These settings are useful for debugging and exploration in notebooks or logs.

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
# ===========================
# Missing Value Imputation
# ===========================
# We begin by copying the original DataFrame to preserve raw data.
# Then, we report the total number of missing values before imputation.

import numpy as np
from sklearn.impute import SimpleImputer

df_filled = df.copy()

print("Starting missing value imputation...")
print(f"Original missing values: {df.isnull().sum().sum()}")

Starting missing value imputation...
Original missing values: 149332775


In [3]:
# def preprocess_flight_data(df):
#     df = df.copy()
    
#     # ========================
#     # 1. Column Renaming
#     # ========================
#     rename_dict = {
#         'Id': 'id',
#         'ranker_id': 'ranker_id',
#         'profileId': 'profile_id',
#         'companyID': 'company_id',
#         'sex': 'user_gender',
#         'nationality': 'user_nationality',
#         'frequentFlyer': 'frequent_flyer_status',
#         'isVip': 'is_vip',
#         'bySelf': 'is_self_booking',
#         'isAccess3D': 'is_access_3d',
#         'corporateTariffCode': 'corporate_tariff_code',
#         'searchRoute': 'search_route',
#         'requestDate': 'request_datetime',
#         'totalPrice': 'total_price',
#         'taxes': 'taxes_amount',
#         'pricingInfo_isAccessTP': 'pricing_policy_compliance',
#         'pricingInfo_passengerCount': 'passenger_count',
#         'selected': 'selected'
#     }
#     df.rename(columns=rename_dict, inplace=True)
    
#     # ========================
#     # 2. Feature Engineering
#     # ========================
    
#     # Trip type from search route
#     df['is_round_trip'] = df['search_route'].str.contains('/').astype(int)
#     print('is_roundtrip')
    
#     # Convert datetimes - handle "NONE" values
#     df['request_datetime'] = pd.to_datetime(df['request_datetime'], errors='coerce')
#     print('requestdatetime')
    
#     # Process legs
#     for leg in [0, 1]:
#         leg_prefix = f'legs{leg}_'
        
#         # Skip leg1 if it doesn't exist
#         if f'{leg_prefix}departureAt' not in df.columns:
#             continue
            
#         # Convert duration to numeric and calculate hours
#         dur_col = f'{leg_prefix}duration'
#         df[dur_col] = pd.to_numeric(df[dur_col], errors='coerce')
#         df[f'legs{leg}_duration_hours'] = df[dur_col] / 60  # Keep original prefix
        
#         # Datetime features - handle "NONE" values
#         for time_type in ['departureAt', 'arrivalAt']:
#             col_name = f'{leg_prefix}{time_type}'
#             # Replace "NONE" with NaT (Not a Time)
#             df[col_name] = df[col_name].replace('NONE', pd.NaT)
#             # Convert to datetime with error handling
#             df[col_name] = pd.to_datetime(df[col_name], errors='coerce')
#             # Create time features without renaming the original column
#             df[f'{leg_prefix}{time_type}_hour'] = df[col_name].dt.hour
#             df[f'{leg_prefix}{time_type}_day_of_week'] = df[col_name].dt.dayofweek
        
#         # Segment processing
#         for seg in range(4):  # 0-3 segments
#             seg_prefix = f'{leg_prefix}segments{seg}_'
            
#             # Skip if segment doesn't exist
#             if f'{seg_prefix}departureFrom_airport_iata' not in df.columns:
#                 continue
                
#             # Airport features
#             df[f'{leg_prefix}seg{seg}_departure_airport'] = df[f'{seg_prefix}departureFrom_airport_iata']
#             df[f'{leg_prefix}seg{seg}_arrival_airport'] = df[f'{seg_prefix}arrivalTo_airport_iata']
#             df[f'{leg_prefix}seg{seg}_arrival_city'] = df[f'{seg_prefix}arrivalTo_airport_city_iata']
            
#             # Airline features
#             marketing = df[f'{seg_prefix}marketingCarrier_code'].astype(str)
#             operating = df[f'{seg_prefix}operatingCarrier_code'].astype(str)
#             df[f'{leg_prefix}seg{seg}_marketing_carrier'] = marketing
#             df[f'{leg_prefix}seg{seg}_operating_carrier'] = operating
#             df[f'{leg_prefix}seg{seg}_carrier_same'] = (marketing == operating).astype(int)
            
#             # Flight details
#             df[f'{leg_prefix}seg{seg}_aircraft_type'] = df[f'{seg_prefix}aircraft_code']
#             df[f'{leg_prefix}seg{seg}_flight_number'] = df[f'{seg_prefix}flightNumber']
            
#             # Convert segment duration to numeric
#             seg_dur_col = f'{seg_prefix}duration'
#             df[seg_dur_col] = pd.to_numeric(df[seg_dur_col], errors='coerce')
#             df[f'{leg_prefix}seg{seg}_duration_hours'] = df[seg_dur_col] / 60
            
#             df[f'{leg_prefix}seg{seg}_seats_available'] = df[f'{seg_prefix}seatsAvailable']
            
#             # Baggage features
#             baggage_col = f'{seg_prefix}baggageAllowance_quantity'
#             df[baggage_col] = pd.to_numeric(df[baggage_col], errors='coerce')
#             df[f'{leg_prefix}seg{seg}_baggage_allowance'] = df[baggage_col]
#             df[f'{leg_prefix}seg{seg}_baggage_type'] = df[f'{seg_prefix}baggageAllowance_weightMeasurementType']
            
#             # Cabin class (mapped to text)
#             cabin_map = {1.0: 'economy', 2.0: 'business', 4.0: 'premium'}
#             df[f'{leg_prefix}seg{seg}_cabin_class'] = df[f'{seg_prefix}cabinClass'].map(cabin_map)

#         print('processed leg')
        
#         # Aggregate leg-level features
#         # Cabin class summary (best cabin in leg)
#         cabin_priority = {'economy': 0, 'premium': 1, 'business': 2}
#         for i in range(4):
#             cabin_col = f'{leg_prefix}seg{i}_cabin_class'
#             if cabin_col in df.columns:
#                 df[f'{leg_prefix}seg{i}_cabin_priority'] = df[cabin_col].map(cabin_priority)
        
#         cabin_cols = [f'{leg_prefix}seg{i}_cabin_priority' for i in range(4) 
#                      if f'{leg_prefix}seg{i}_cabin_priority' in df.columns]
#         if cabin_cols:
#             df[f'{leg_prefix}best_cabin'] = df[cabin_cols].max(axis=1).map({v:k for k,v in cabin_priority.items()})
        
#         # Total baggage allowance
#         baggage_cols = [f'{leg_prefix}seg{i}_baggage_allowance' for i in range(4)
#                        if f'{leg_prefix}seg{i}_baggage_allowance' in df.columns]
#         if baggage_cols:
#             df[f'{leg_prefix}total_baggage'] = df[baggage_cols].sum(axis=1)

#         print('processed baggage and cabin')
    
#     # Process rules
#     rule_types = {0: 'cancellation', 1: 'exchange'}
#     for rule_num, rule_name in rule_types.items():
#         prefix = f'miniRules{rule_num}_'
#         df[f'{rule_name}_penalty_amount'] = pd.to_numeric(df[f'{prefix}monetaryAmount'], errors='coerce')
#         df[f'{rule_name}_penalty_percentage'] = pd.to_numeric(df[f'{prefix}percentage'], errors='coerce')
#         df[f'{rule_name}_allowed'] = (pd.to_numeric(df[f'{prefix}statusInfos'], errors='coerce') != 0).astype(int)

#     print('processed rule type')
    
#     # Pricing features
#     df['base_fare'] = df['total_price'] - df['taxes_amount']
#     df['price_per_passenger'] = df['total_price'] / df['passenger_count'].clip(lower=1)
#     print('processed pricing feature')
    
#     # Time until departure - FIXED: use original column name
#     if 'legs0_departureAt' in df.columns:
#         df['days_until_departure'] = (df['legs0_departureAt'] - df['request_datetime']).dt.days
#     print('processed time until departure')
#     # ========================
#     # 3. Column Removal
#     # ========================
    
#     # Remove original nested columns
#     cols_to_remove = []
    
#     # Remove original leg columns we don't need anymore
#     cols_to_remove.extend([
#         'legs0_duration', 'legs1_duration',
#         'search_route'  # We extracted is_round_trip
#     ])
    
#     # Remove all original segment columns
#     for col in df.columns:
#         if col.startswith('legs') and any(part in col for part in ['segments', 'marketingCarrier', 'operatingCarrier', 
#                                                                 'aircraft_code', 'flightNumber', 'duration', 
#                                                                 'baggageAllowance', 'cabinClass']):
#             cols_to_remove.append(col)
    
#     # Remove miniRules columns
#     cols_to_remove.extend([f'miniRules{i}_{part}' for i in [0,1] 
#                           for part in ['monetaryAmount', 'percentage', 'statusInfos']])
    
#     # Remove columns that don't exist in the DataFrame
#     cols_to_remove = [col for col in cols_to_remove if col in df.columns]
    
#     # Drop the columns
#     df.drop(columns=cols_to_remove, inplace=True, errors='ignore')
    
#     # ========================
#     # 4. Final Processing
#     # ========================
    
#     # Convert categorical columns
#     categorical_cols = [
#         'user_gender', 'user_nationality', 'frequent_flyer_status',
#         'corporate_tariff_code'
#     ]
#     # Only convert existing columns
#     categorical_cols = [col for col in categorical_cols if col in df.columns]
#     for col in categorical_cols:
#         df[col] = df[col].astype('category')
    
#     # Fill missing values for one-way trips
#     if 'legs1_departureAt' in df.columns:
#         leg1_cols = [col for col in df.columns if col.startswith('legs1')]
#         # Create a dictionary for fill values
#         fill_dict = {
#             'legs1_duration_hours': 0,
#             'legs1_total_baggage': 0,
#             'legs1_best_cabin': 'none'
#         }
#         # Only include columns that actually exist in the DataFrame
#         fill_dict = {k: v for k, v in fill_dict.items() if k in df.columns}
#         df[leg1_cols] = df[leg1_cols].fillna(fill_dict)
    
#     # Fill other NaNs with appropriate defaults
#     numeric_cols = df.select_dtypes(include=np.number).columns
#     df[numeric_cols] = df[numeric_cols].fillna(0)
    
#     # Fill datetime NaNs with a default date (e.g., 1970-01-01) or keep as NaT
#     datetime_cols = df.select_dtypes(include=['datetime64[ns]']).columns
#     # Option: Keep as NaT or fill with a specific date
#     # df[datetime_cols] = df[datetime_cols].fillna(pd.Timestamp('1970-01-01'))
    
#     # Fill object columns with 'missing'
#     object_cols = df.select_dtypes(include='object').columns
#     df[object_cols] = df[object_cols].fillna('missing')
    
#     return df

# # Load your dataset
# # df = pd.read_csv('your_data.csv')

# # Apply preprocessing
# processed_df = preprocess_flight_data(df_filled)

# # Check for remaining type issues
# print(processed_df.dtypes)

# # Save processed data
# # processed_df.to_csv('processed_flight_data.csv', index=False)

In [2]:
# Corporate Tariff Code (Int64 dtype) - Fill with pandas NA for integer columns
# Int64 is pandas nullable integer type, so we use pd.NA instead of string
# Convert to regular int64 and use -1 as indicator for "no corporate tariff"
df_filled['corporateTariffCode'] = df_filled['corporateTariffCode'].fillna(-1).astype('int64')
print("✓ Filled corporateTariffCode with -1 (converted to int64)")

In [10]:
# Travel Policy Access (float64) - fill with 0.0 (non-compliant/not applicable)
df_filled['pricingInfo_isAccessTP'].fillna(0.0, inplace=True)
print("✓ Filled pricingInfo_isAccessTP with 0.0")

/tmp/ipykernel_10/3030471879.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled['pricingInfo_isAccessTP'].fillna(0.0, inplace=True)


✓ Filled pricingInfo_isAccessTP with 0.0


In [11]:
# Frequent Flyer Status (object dtype) - Fill with NONE for users without FF status
df_filled['frequentFlyer'].fillna('NONE', inplace=True)
print("Filled frequentFlyer with NONE")

/tmp/ipykernel_10/1784150114.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled['frequentFlyer'].fillna('NONE', inplace=True)


Filled frequentFlyer with NONE


In [12]:
# For airport codes (object dtype), we use mode imputation since forward fill
# with object columns can be unreliable. SimpleImputer handles object dtypes well

airport_cols_to_fill = [
    'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_arrivalTo_airport_iata', 
    'legs1_segments0_departureFrom_airport_iata'
]

# using simpleImputer for object columns - it handles NaN in object dtypes properly
for col in airport_cols_to_fill:
    if df_filled[col].isnull().sum() > 0:
        # Create imputer for this specific column
        imputer = SimpleImputer(strategy='most_frequent')
        # Reshape for sklearn (needs 2D array) and flatten back
        df_filled[col] = imputer.fit_transform(df_filled[[col]]).ravel()

print("✓ Filled airport codes using mode imputation")


✓ Filled airport codes using mode imputation


In [13]:
# Return Flight Fields (One Way vs Round Trip) 
# These are datetime like objects, so we use string indicator instead of 0
return_flight_object_cols = [
    'legs1_arrivalAt', 'legs1_departureAt', 'legs1_duration'
]

for col in return_flight_object_cols:
    df_filled[col].fillna('NONE', inplace=True)

print("✓ Filled return flight timing fields with 'NONE'")

/tmp/ipykernel_10/886912719.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled[col].fillna('NONE', inplace=True)


✓ Filled return flight timing fields with 'NONE'


In [14]:
# Get all segment columns and separate by data type
segment_cols = [col for col in df_filled.columns if 'segments' in col]

# Separate columns by dtype for appropriate filling
object_segment_cols = []
float_segment_cols = []

for col in segment_cols:
    if df_filled[col].dtype == 'object':
        object_segment_cols.append(col)
    elif df_filled[col].dtype == 'float64':
        float_segment_cols.append(col)

# Fill object segment fields with NONE (no connection exists)
for col in object_segment_cols:
    df_filled.fillna({col: 'NONE'}, inplace=True)

# Fill float segment fields with 0.0 (no connection/duration/seats/etc)
for col in float_segment_cols:
    df_filled.fillna({col: '0.0'}, inplace=True)

print(f"✓ Filled {len(object_segment_cols)} object segment columns with 'NONE'")
print(f"✓ Filled {len(float_segment_cols)} float segment columns with 0.0")

✓ Filled 64 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0


In [15]:
# All miniRules columns are float64, so we use appropriate float values

# Monetary penalties (float64) - Fill with 0.0 (no monetary penalty)
df_filled['miniRules0_monetaryAmount'].fillna(0.0, inplace=True)  # Cancellation
df_filled['miniRules1_monetaryAmount'].fillna(0.0, inplace=True)  # Exchange

# Percentage penalties (float64) - Fill with 100.0 (100% penalty = no refund/exchange)
# This is more conservative and realistic for restrictive tickets
df_filled['miniRules0_percentage'].fillna(100.0, inplace=True)    # Cancellation
df_filled['miniRules1_percentage'].fillna(100.0, inplace=True)    # Exchange

# Status information (float64) - Fill with 0.0 (not allowed)
df_filled['miniRules0_statusInfos'].fillna(0.0, inplace=True)     # Cancellation
df_filled['miniRules1_statusInfos'].fillna(0.0, inplace=True)     # Exchange

print("✓ Filled cancellation and exchange rules with appropriate float values")

/tmp/ipykernel_10/1464864012.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_filled['miniRules0_monetaryAmount'].fillna(0.0, inplace=True)  # Cancellation
/tmp/ipykernel_10/1464864012.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method

✓ Filled cancellation and exchange rules with appropriate float values


In [16]:
# Remaining missing values
remaining_missing = df_filled.isnull().sum()
total_remaining = remaining_missing.sum()

print(f"\n=== IMPUTATION SUMMARY ===")
print(f"Original missing values: {df.isnull().sum().sum():,}")
print(f"Remaining missing values: {total_remaining:,}")
print(f"Successfully filled: {df.isnull().sum().sum() - total_remaining:,} values")

# Show columns that still have missing values (if any)
if total_remaining > 0:
    print(f"\nColumns still with missing values:")
    print(remaining_missing[remaining_missing > 0])
else:
    print("\n🎉 All missing values have been successfully filled!")


=== IMPUTATION SUMMARY ===
Original missing values: 1,352,535,929
Remaining missing values: 0
Successfully filled: 1,352,535,929 values

🎉 All missing values have been successfully filled!


In [17]:
# Data quality checks 
print(f"\n=== DATA QUALITY CHECKS ===")
print(f"DataFrame shape: {df_filled.shape}")

# Check data types after imputation
print(f"\nData types after imputation:")
dtype_changes = []
for col in df_filled.columns:
    if df[col].dtype != df_filled[col].dtype:
        dtype_changes.append(f"{col}: {df[col].dtype} → {df_filled[col].dtype}")

if dtype_changes:
    print("Data type changes:")
    for change in dtype_changes[:5]:  # Show first 5 changes
        print(f"  {change}")
    if len(dtype_changes) > 5:
        print(f"  ... and {len(dtype_changes) - 5} more")
else:
    print("No data type changes occurred")


=== DATA QUALITY CHECKS ===
DataFrame shape: (18145372, 127)

Data types after imputation:
Data type changes:
  corporateTariffCode: Int64 → int64
  legs0_segments0_baggageAllowance_quantity: float64 → object
  legs0_segments0_baggageAllowance_weightMeasurementType: float64 → object
  legs0_segments0_seatsAvailable: float64 → object
  legs0_segments1_baggageAllowance_quantity: float64 → object
  ... and 27 more


In [18]:
# Check filled values
print(f"\nSample of filled values:")
print(f"corporateTariffCode range: {df_filled['corporateTariffCode'].min()} to {df_filled['corporateTariffCode'].max()}")
print(f"frequentFlyer unique values: {df_filled['frequentFlyer'].nunique()}")
print(f"miniRules0_percentage range: {df_filled['miniRules0_percentage'].min():.1f} to {df_filled['miniRules0_percentage'].max():.1f}")


Sample of filled values:
corporateTariffCode range: -1 to 181
frequentFlyer unique values: 372
miniRules0_percentage range: 0.0 to 100.0


In [19]:
# Check for any unexpected data types
print(f"\nFinal data types summary:")
print(df_filled.dtypes.value_counts())


Final data types summary:
object            104
float64            10
int64               8
bool                4
datetime64[ns]      1
Name: count, dtype: int64


In [20]:
print(f"\n=== IMPORTANT NOTES ===")
print("Data type considerations:")
print("• Int64 → int64: Converted nullable integer to regular integer with -1 as null indicator")
print("• object columns: Used 'NONE' as null indicator for categorical/string data")  
print("• float64 columns: Used 0.0 for numeric nulls, 100.0 for percentage penalties")
print("• datetime objects: Used 'NONE' string indicator (could convert to datetime later)")

print(f"\n✅ Dataset is ready for further analysis!")
print(f"Use 'df_filled' for your clean dataset with no missing values")


=== IMPORTANT NOTES ===
Data type considerations:
• Int64 → int64: Converted nullable integer to regular integer with -1 as null indicator
• object columns: Used 'NONE' as null indicator for categorical/string data
• float64 columns: Used 0.0 for numeric nulls, 100.0 for percentage penalties
• datetime objects: Used 'NONE' string indicator (could convert to datetime later)

✅ Dataset is ready for further analysis!
Use 'df_filled' for your clean dataset with no missing values


In [21]:
print("\n1. Converting datetime columns...")
datetime_columns = [
    'legs0_arrivalAt', 'legs0_departureAt',
    'legs1_arrivalAt', 'legs1_departureAt'
]

# Don't convert 'NONE' to NaT, keep as string or replace with valid datetime
for col in datetime_columns:
    if col in df_filled.columns:
        print(f"Processing {col}...")
        # Option 1: Skip rows with 'NONE' entirely
        mask = df_filled[col] != 'NONE'
        df_filled.loc[mask, col] = pd.to_datetime(df_filled.loc[mask, col], errors='coerce')
        # Leave 'NONE' as string (don't convert to datetime)

print("Datetime conversion completed")


1. Converting datetime columns...
Processing legs0_arrivalAt...
Processing legs0_departureAt...
Processing legs1_arrivalAt...
Processing legs1_departureAt...
Datetime conversion completed


In [22]:
print("\n3. Optimizing categorical variables...")

# Convert string categories to categorical dtype for memory efficiency
categorical_columns = [
    'frequentFlyer', 'searchRoute', 'ranker_id'
]

# Add airport and airline code columns
airport_airline_cols = [col for col in df_filled.columns 
                       if any(keyword in col for keyword in ['iata', 'code', 'flightNumber']) 
                       and df_filled[col].dtype == 'object']
categorical_columns.extend(airport_airline_cols)

for col in categorical_columns:
    if col in df_filled.columns:
        # Convert to categorical if it has reasonable number of unique values
        unique_count = df_filled[col].nunique()
        total_count = len(df_filled)
        
        # Convert to categorical if less than 50% unique values (arbitrary threshold)
        if unique_count / total_count < 0.5:
            df_filled[col] = df_filled[col].astype('category')
            print(f"   ✓ {col}: {unique_count} categories")
        else:
            print(f"   → {col}: Too many unique values ({unique_count}), keeping as object")



3. Optimizing categorical variables...
   ✓ frequentFlyer: 372 categories
   ✓ searchRoute: 5769 categories
   ✓ ranker_id: 105539 categories
   ✓ legs0_segments0_aircraft_code: 108 categories
   ✓ legs0_segments0_arrivalTo_airport_city_iata: 500 categories
   ✓ legs0_segments0_arrivalTo_airport_iata: 534 categories
   ✓ legs0_segments0_departureFrom_airport_iata: 394 categories
   ✓ legs0_segments0_flightNumber: 7531 categories
   ✓ legs0_segments0_marketingCarrier_code: 164 categories
   ✓ legs0_segments0_operatingCarrier_code: 220 categories
   ✓ legs0_segments1_aircraft_code: 103 categories
   ✓ legs0_segments1_arrivalTo_airport_city_iata: 425 categories
   ✓ legs0_segments1_arrivalTo_airport_iata: 476 categories
   ✓ legs0_segments1_departureFrom_airport_iata: 455 categories
   ✓ legs0_segments1_flightNumber: 6577 categories
   ✓ legs0_segments1_marketingCarrier_code: 153 categories
   ✓ legs0_segments1_operatingCarrier_code: 207 categories
   ✓ legs0_segments2_aircraft_code: 82 

In [23]:
print("\n6. Performing data validation...")

# Check for any remaining missing values
missing_check = df_filled.isnull().sum()
if missing_check.sum() > 0:
    print(f"   ⚠️  Warning: {missing_check.sum()} missing values remain:")
    print(missing_check[missing_check > 0].head())
else:
    print("   ✓ No missing values found")



6. Performing data validation...
   ✓ No missing values found


In [24]:
# Check for infinite values
inf_cols = []
for col in df_filled.select_dtypes(include=[np.number]).columns:
    if np.isinf(df_filled[col]).any():
        inf_cols.append(col)

if inf_cols:
    print(f"   ⚠️  Warning: Infinite values found in: {inf_cols}")
    # Replace inf with large finite values
    for col in inf_cols:
        df_filled[col] = df_filled[col].replace([np.inf, -np.inf], [df_filled[col].max()*10, df_filled[col].min()*10])
    print("   ✓ Infinite values replaced")
else:
    print("   ✓ No infinite values found")

# Check data ranges
print("   Data range validation:")
print(f"   • Total price range: ${df_filled['totalPrice'].min():.2f} - ${df_filled['totalPrice'].max():.2f}")
print(f"   • Flight duration range: {df_filled['legs0_duration'].min():} - {df_filled['legs0_duration'].max():} minutes")
print(f"   • Passenger count range: {df_filled['pricingInfo_passengerCount'].min()} - {df_filled['pricingInfo_passengerCount'].max()}")


   ✓ No infinite values found
   Data range validation:
   • Total price range: $770.00 - $9944355.00
   • Flight duration range: 00:30:00 - 6.03:15:00 minutes
   • Passenger count range: 1 - 1


In [ ]:
print("\n7. Optimizing memory usage...")

# Calculate memory usage before and after
memory_before = df_filled.memory_usage(deep=True).sum() / (1024**2)  # MB

# Additional memory optimizations for float columns
float_cols = df_filled.select_dtypes(include=['float64']).columns
for col in float_cols:
    if col not in ['totalPrice', 'taxes']:  # Keep price columns as float32
        # Check if we can downcast to float32
        col_min, col_max = df_filled[col].min(), df_filled[col].max()
        if col_min >= np.finfo(np.float32).min and col_max <= np.finfo(np.float32).max:
            df_filled[col] = df_filled[col].astype('float32')

memory_after = df_filled.memory_usage(deep=True).sum() / (1024**2)  # MB
memory_saved = memory_before - memory_after

print(f"   Memory usage: {memory_before:.1f} MB → {memory_after:.1f} MB")
print(f"   Memory saved: {memory_saved:.1f} MB ({memory_saved/memory_before*100:.1f}%)")

In [25]:
# For leg durations
df_filled = df_filled.rename(columns={
    'legs0_duration_minutes': 'leg0_total_minutes',
    'legs1_duration_minutes': 'leg1_total_minutes'
})

# For segment durations - remove the redundant suffix
for col in df_filled.columns:
    if '_minutes_minutes' in col:
        new_name = col.replace('_minutes_minutes', '_minutes')
        df_filled = df_filled.rename(columns={col: new_name})

In [ ]:
# Show all columns in current session
pd.set_option('display.max_columns', None)

# Now when you view your DataFrame, all columns will show
print(df.columns.tolist())  # List of all columns


In [ ]:
# Group Analysis (ranker_id)
group_sizes = df_filled.groupby("ranker_id").size().reset_index(name="group_size")
print(group_sizes["group_size"].describe().to_string())

In [ ]:
# Use pandas filtering
groups_over_10 = group_sizes[group_sizes["group_size"] > 10]
percentage_over_10 = (len(groups_over_10) / len(group_sizes)) * 100
print(f"\nPercentage of queries (ranker_id) with >10 options: {percentage_over_10:.2f}%")

In [ ]:
# Check results
print(f"Original columns: {len(df_filled.columns)}")
print(f"Engineered columns: {len(df_engineered.columns)}")
print(f"NaN values: {df_engineered.isna().sum().sum()}")

In [26]:
# ===========================
# Data Preparation for Ranking Model (LightGBM + Optuna)
# ===========================
# This block handles data preprocessing and feature engineering for training a LightGBM ranking model.
# The process includes:
#
# 🔹 Group-aware train/validation split using GroupShuffleSplit (grouped by `ranker_id`).
# 🔹 Separation of target variable (`selected`) from features.
# 🔹 Deduplication of feature columns to avoid model training issues.
# 🔹 Extraction and transformation of datetime features with:
#     - Hour, minute, weekday, and weekend flags.
#     - Cyclical encoding (sin/cos) for hourly patterns — helpful for capturing temporal behavior.
#
# 👉 The model will later use group info (`ranker_id`) for learning to rank

import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit
# from sklearn.preprocessing import LabelEncoder
import time

# Enhanced datetime feature extraction
def extract_datetime_features(df, time_cols):
    for col in time_cols:
        df[col + '_hour'] = df[col].dt.hour
        df[col + '_minute'] = df[col].dt.minute
        df[col + '_dayofweek'] = df[col].dt.dayofweek
        df[col + '_is_weekend'] = (df[col].dt.dayofweek >= 5).astype(int)
        # Add cyclic encoding for time features
        df[col + '_hour_sin'] = np.sin(2 * np.pi * df[col + '_hour']/24)
        df[col + '_hour_cos'] = np.cos(2 * np.pi * df[col + '_hour']/24)
    return df.drop(columns=time_cols)

# ----------------------------------
# Metric: hitrate_at_3
# ----------------------------------
# Custom evaluation metric for ranking quality.
# It checks if the correct flight was within the top-3 predictions in each session group.
# 
# This aligns with typical top-K recommendation evaluation strategies and helps measure practical impact.

def hitrate_at_3(y_true, y_pred, group_sizes, k=3):
    group_ids = np.repeat(np.arange(len(group_sizes)), group_sizes)

    df = pd.DataFrame({
        'group': group_ids,
        'true': y_true,
        'pred': y_pred
    })

    hitrate = (
        df.groupby('group')
        .apply(lambda g: g.nlargest(min(k, len(g)), 'pred')['true'].max())
        .mean()
    )

    return hitrate if not np.isnan(hitrate) else 0.0

print("Preparing data...")
session_groups = df_filled['ranker_id'].values
X = df_filled.drop(columns=['selected', 'ranker_id'])
y = df_filled['selected'].values

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_idx, val_idx = next(gss.split(X, y, groups=session_groups))

X_train, X_val = X.iloc[train_idx].copy(), X.iloc[val_idx].copy()
y_train, y_val = y[train_idx], y[val_idx]

train_groups = df_filled.iloc[train_idx].groupby('ranker_id').size().values
val_groups = df_filled.iloc[val_idx].groupby('ranker_id').size().values

# Get ranker_id for group calculations
train_ranker_ids = df_filled.iloc[train_idx]['ranker_id'].values
val_ranker_ids = df_filled.iloc[val_idx]['ranker_id'].values

# Drop duplicate columns
X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_val = X_val.loc[:, ~X_val.columns.duplicated()]

# Handle datetime features
datetime_cols = X_train.select_dtypes(include=['datetime64']).columns.tolist()
if datetime_cols:
    print(f"Processing datetime columns: {datetime_cols}")
    X_train = extract_datetime_features(X_train, datetime_cols)
    X_val = extract_datetime_features(X_val, datetime_cols)


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Preparing data...


/tmp/ipykernel_10/299905486.py:49: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_groups = df_filled.iloc[train_idx].groupby('ranker_id').size().values
/tmp/ipykernel_10/299905486.py:50: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  val_groups = df_filled.iloc[val_idx].groupby('ranker_id').size().values


Processing datetime columns: ['requestDate']


In [ ]:
X_train.shape

In [27]:
# ===========================
# Categorical Feature Encoding (Target Encoding)
# ===========================
# This step encodes categorical variables using **target encoding**, which replaces each category
# with the mean target value for that category (with smoothing to avoid overfitting).
#
# 🔹 We use `category_encoders.TargetEncoder` with smoothing=1.0 to prevent data leakage and high variance on rare categories.
# 🔹 The encoder is fitted **only on training data** to ensure validation remains untouched.
# 🔹 This method is especially helpful when dealing with high-cardinality categories 
#
# 👉 Always verify that target encoding doesn’t introduce leakage by accidentally using label information from the validation/test set.

import category_encoders as ce

# Handle categorical features with target encoding
print("Processing categorical features...")
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
if cat_cols:
    print(f"Categorical columns: {cat_cols}")
    te = ce.TargetEncoder(cols=cat_cols, smoothing=1.0)
    X_train[cat_cols] = te.fit_transform(X_train[cat_cols], y_train)
    X_val[cat_cols] = te.transform(X_val[cat_cols])


Processing categorical features...
Categorical columns: ['frequentFlyer', 'legs0_arrivalAt', 'legs0_departureAt', 'legs0_duration', 'legs0_segments0_aircraft_code', 'legs0_segments0_arrivalTo_airport_city_iata', 'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_baggageAllowance_quantity', 'legs0_segments0_baggageAllowance_weightMeasurementType', 'legs0_segments0_departureFrom_airport_iata', 'legs0_segments0_duration', 'legs0_segments0_flightNumber', 'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code', 'legs0_segments0_seatsAvailable', 'legs0_segments1_aircraft_code', 'legs0_segments1_arrivalTo_airport_city_iata', 'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_baggageAllowance_quantity', 'legs0_segments1_baggageAllowance_weightMeasurementType', 'legs0_segments1_cabinClass', 'legs0_segments1_departureFrom_airport_iata', 'legs0_segments1_duration', 'legs0_segments1_flightNumber', 'legs0_segments1_marketingCarrier_code', 'legs0_segments1_op

In [ ]:
X_train.shape

In [1]:
# ===========================
# Hyperparameter Tuning: LGBMRanker with Optuna
# ===========================
# We use Optuna to optimize a LightGBM LambdaRank model for ranking flight options.
# The objective is to **maximize HitRate@3**, a custom evaluation metric aligned with the business goal
# (whether the correct option appears in the top 3 predicted).
#
# 🔹 `objective`: 'lambdarank' optimizes pairwise ranking with NDCG.
# 🔹 Optuna searches over:
#     - learning_rate, num_leaves, regularization (L1/L2), and data subsampling params
# 🔹 Uses Group data (`ranker_id`) via LightGBM's `group` and `eval_group` arguments.
# 🔹 `n_estimators` is set high (2000) to allow early stopping to find optimal iteration count.
#
# ⚠️ Important:
#     - We ensure reproducibility with a fixed seed (TPESampler).
#     - Unpromising trials are pruned using `MedianPruner` to speed up tuning.
#     - Categorical columns are passed explicitly (if available).
#
# 👉 Final best parameters and score are printed after optimization completes.

# ----------------------------------
# Running the Optuna Study
# ----------------------------------
# - direction='maximize': because higher HitRate@3 is better
# - n_trials=50: can increase for better search (if time/memory allows)
# - timeout=24h: ensures overnight runs terminate
# - show_progress_bar=True: helpful in notebooks/CLI

from lightgbm import LGBMRanker

def objective(trial):
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'ndcg_eval_at': [3],
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 16, 128, step=8),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 200, step=10),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 1.0, step=0.1),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 1.0, step=0.1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
        'force_row_wise': True,
        'random_state': 42,
        'verbosity': -1,
        'n_estimators': 2000,  # Increased to ensure early stopping decides actual trees
    }

    model = LGBMRanker(**params)

    model.fit(
        X_train, y_train,
        group=train_groups,
        eval_set=[(X_val, y_val)],
        eval_group=[val_groups],
        categorical_feature=cat_cols
    )

    y_val_pred = model.predict(X_val)
    hitrate = hitrate_at_3(y_val, y_val_pred, val_groups)
    return hitrate

# Configure and run Optuna study
study = optuna.create_study(
    direction='maximize',
    sampler=optuna.samplers.TPESampler(seed=42),  # For reproducible optimization
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=10)  # Prune unpromising trials
)
study.optimize(objective, n_trials=50, timeout=24*3600, show_progress_bar=True)

# Output results
print("\n" + "="*50)
print(f"✅ Best HitRate@3: {study.best_value:.4f}")
print("✅ Best Params:", study.best_params)
print("="*50)
for key, value in study.best_params.items():
    print(f"  - {key}: {value}")
print("="*50)

In [28]:
# ===========================
# Final Model Training with Best Parameters (LGBMRanker)
# ===========================
# Trains the final LightGBM LambdaRank model using the best hyperparameters obtained from Optuna.
#
# 🔹 `best_params` holds the tuned config, manually selected or copied from the best trial.
# 🔹 We use LightGBM's built-in early stopping to prevent overfitting, monitoring NDCG@3 on validation data.
# 🔹 Group information (`ranker_id`) is passed explicitly via `group` and `eval_group`.
#
# ✅ Model is saved using joblib for portability and reusability.

# 2. Set best parameters from Optuna
best_params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [3],
    'boosting_type': 'gbdt',
    'n_estimators': 1000,
    'learning_rate': 0.05,
    'num_leaves': 63,
    'max_depth': -1,  # No limit
    'min_child_samples': 20,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'max_bin': 255,
    'random_state': 42,
    'n_jobs': -1,
    'importance_type': 'gain',
    'verbose': -1,
    'seed': 42
}

# train_set = lgb.Dataset(X_train, label=y_train, group=train_groups, categorical_feature=cat_cols)
# val_set = lgb.Dataset(X_val, label=y_val, group=val_groups, reference=train_set, categorical_feature=cat_cols)

from lightgbm import LGBMRanker

# 1. Define model
ranker = LGBMRanker(
    **best_params,
)

# 2. Fit the model
ranker.fit(
    X_train,
    y_train,
    group=train_groups,
    eval_set=[(X_val, y_val)],
    eval_group=[val_groups],
    # eval_at=[3],
    # categorical_feature=cat_cols,
    callbacks=[
        lgb.early_stopping(stopping_rounds=50, verbose=True),
        lgb.log_evaluation(period=100)
    ]
)
import joblib

# Save the model to a file
joblib.dump(ranker, "lgb_ranker_model3.pkl")
print("✅ Model saved to lgb_ranker_model.pkl")


/usr/local/lib/python3.10/site-packages/lightgbm/sklearn.py:861: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")


Training until validation scores don't improve for 50 rounds
[100]	valid_0's ndcg@3: 0.961296
[200]	valid_0's ndcg@3: 0.962492
[300]	valid_0's ndcg@3: 0.962976
[400]	valid_0's ndcg@3: 0.963245
[500]	valid_0's ndcg@3: 0.963473
[600]	valid_0's ndcg@3: 0.963649
[700]	valid_0's ndcg@3: 0.963787
[800]	valid_0's ndcg@3: 0.963898
[900]	valid_0's ndcg@3: 0.964114
[1000]	valid_0's ndcg@3: 0.964242
Did not meet early stopping. Best iteration is:
[989]	valid_0's ndcg@3: 0.964243
✅ Model saved to lgb_ranker_model.pkl


In [ ]:
# # # Use your existing hitrate_at_3 function
# # def hitrate_at_3(y_true, y_pred):
# #     """
# #     Calculate hitrate@3 metric.
# #     Args:
# #         y_true: List of sets/lists containing ground truth relevant items for each user.
# #         y_pred: List of lists of recommended items (ranked in order) for each user
# #     Returns:
# #         hitrate@3 score (float) in the range [0, 1].
# #     """
# #     total_hits = 0
# #     num_users = len(y_true)
# #     for true_items, preds in zip(y_true, y_pred):
# #         true_set = set(true_items)
# #         if any(item in true_set for item in preds[:3]):
# #             total_hits += 1
# #     return total_hits / num_users

# def calculate_hit_rate_at_3(df_preds_with_true_and_rank):
#     """
#     Calculates HitRate@3.
#     df_preds_with_true_and_rank must have:
#         - 'ranker_id'
#         - 'selected' (true binary target, 1 for chosen)
#         - 'predicted_rank' (rank assigned by the model, 1 is best)
#     """
#     hits = 0
#     valid_queries_count = 0
    
#     for ranker_id, group in df_preds_with_true_and_rank.groupby('ranker_id'):
#         if len(group) <= 10:
#             continue  # Skip groups with 10 or fewer options as per competition rules
        
#         valid_queries_count += 1
        
#         true_selected_item = group[group['selected'] == 1]
        
#         if not true_selected_item.empty:
#             # Get the rank of the true selected item
#             rank_of_true_item = true_selected_item.iloc[0]['predicted_rank']
#             if rank_of_true_item <= 3:
#                 hits += 1
#         # else:
#             # This shouldn't happen in validation if data is prepared correctly from train
#             # print(f"Warning: No selected item found for ranker_id {ranker_id} in HitRate calculation.")
            
#     if valid_queries_count == 0:
#         return 0.0
#     return hits / valid_queries_count

In [29]:

# ===========================
# Preprocessing Test Data (Same as Training Pipeline)
# ===========================
# Applies the same preprocessing steps used for training:
# 🔹 Datetime feature extraction
# 🔹 Target/categorical encoding using pre-fitted encoder
# 🔹 Column alignment and cleanup
#
# 👉 Ensures consistency between training and inference data.

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load test data
test_path = "/kaggle/input/aeroclub-recsys-cup-2025-1/test.parquet"

import numpy as np
from sklearn.impute import SimpleImputer
import duckdb
import pandas as pd

train_path = "/kaggle/input/aeroclub-recsys-cup-2025-1/train.parquet"
test_path = "/kaggle/input/aeroclub-recsys-cup-2025-1/test.parquet"

con = duckdb.connect()

# # Correct session sampling with minimum flight options
# query = f"""
#     WITH valid_sessions AS (
#         SELECT ranker_id
#         FROM parquet_scan('{test_path}')
#         GROUP BY ranker_id
#         HAVING COUNT(*) >= 3  -- Sessions with at least 10 flight options
#     ),
#     sampled_sessions AS (
#         SELECT ranker_id
#         FROM valid_sessions
#         USING SAMPLE 55000  -- Sample 5000 sessions (adjust based on memory)
#     )
#     SELECT t.*
#     FROM parquet_scan('{test_path}') t
#     JOIN sampled_sessions s ON t.ranker_id = s.ranker_id
# """

query = f"""
    SELECT *
    FROM parquet_scan('{test_path}')
"""

# Execute query and convert to DataFrame
df = con.query(query).to_df()
con.close()

df_test = df

# Check session stats
session_counts = df_test['ranker_id'].value_counts()
print(f"Loaded {len(df_test)} rows from {len(session_counts)} sessions")
print(f"Min options per session: {session_counts.min()}")
print(f"Max options per session: {session_counts.max()}")

print("Starting missing value imputation...")
print(f"Original missing values: {df_test.isnull().sum().sum()}")

df_test['corporateTariffCode'] = df_test['corporateTariffCode'].fillna(-1).astype('int64')
print("✓ Filled corporateTariffCode with -1 (converted to int64)")

df_test['pricingInfo_isAccessTP'].fillna(0.0, inplace=True)
print("✓ Filled pricingInfo_isAccessTP with 0.0")

df_test['frequentFlyer'].fillna('NONE', inplace=True)
print("Filled frequentFlyer with NONE")

airport_cols_to_fill = [
'legs0_segments0_arrivalTo_airport_city_iata',
'legs0_segments0_arrivalTo_airport_iata',
'legs1_segments0_departureFrom_airport_iata'
]

return_flight_object_cols = [
'legs1_arrivalAt', 'legs1_departureAt', 'legs1_duration'
]

for col in return_flight_object_cols:
    df_test[col].fillna('NONE', inplace=True)

    print("✓ Filled return flight timing fields with 'NONE'")

# Get all segment columns and separate by data type

segment_cols = [col for col in df_test.columns if 'segments' in col]

# Separate columns by dtype for appropriate filling

object_segment_cols = []
float_segment_cols = []

for col in segment_cols:
    if df_test[col].dtype == 'object':
        object_segment_cols.append(col)
    elif df_test[col].dtype == 'float64':
        float_segment_cols.append(col)

# Fill object segment fields with NONE (no connection exists)

for col in object_segment_cols:
    df_test.fillna({col: 'NONE'}, inplace=True)

# Fill float segment fields with 0.0 (no connection/duration/seats/etc)

for col in float_segment_cols:
    df_test.fillna({col: '0.0'}, inplace=True)

    print(f"✓ Filled {len(object_segment_cols)} object segment columns with 'NONE'")
    print(f"✓ Filled {len(float_segment_cols)} float segment columns with 0.0")

# All miniRules columns are float64, so we use appropriate float values

# Monetary penalties (float64) - Fill with 0.0 (no monetary penalty)

df_test['miniRules0_monetaryAmount'].fillna(0.0, inplace=True)  # Cancellation
df_test['miniRules1_monetaryAmount'].fillna(0.0, inplace=True)  # Exchange

# Percentage penalties (float64) - Fill with 100.0 (100% penalty = no refund/exchange)

# This is more conservative and realistic for restrictive tickets

df_test['miniRules0_percentage'].fillna(100.0, inplace=True)    # Cancellation
df_test['miniRules1_percentage'].fillna(100.0, inplace=True)    # Exchange

# Status information (float64) - Fill with 0.0 (not allowed)

df_test['miniRules0_statusInfos'].fillna(0.0, inplace=True)     # Cancellation
df_test['miniRules1_statusInfos'].fillna(0.0, inplace=True)     # Exchange

print("✓ Filled cancellation and exchange rules with appropriate float values")

print("\n1. Converting datetime columns...")
datetime_columns = [
'legs0_arrivalAt', 'legs0_departureAt',
'legs1_arrivalAt', 'legs1_departureAt'
]

# Don't convert 'NONE' to NaT, keep as string or replace with valid datetime

for col in datetime_columns:
    if col in df_test.columns:
        print(f"Processing {col}...")
        # Option 1: Skip rows with 'NONE' entirely
        mask = df_test[col] != 'NONE'
        df_test.loc[mask, col] = pd.to_datetime(df_test.loc[mask, col], errors='coerce')
        # Leave 'NONE' as string (don't convert to datetime)

print("Datetime conversion completed")

print("\n3. Optimizing categorical variables...")

# Convert string categories to categorical dtype for memory efficiency
categorical_columns = [
    'frequentFlyer', 'searchRoute', 'ranker_id'
]

# Add airport and airline code columns
airport_airline_cols = [col for col in df_test.columns 
                       if any(keyword in col for keyword in ['iata', 'code', 'flightNumber']) 
                       and df_test[col].dtype == 'object']
categorical_columns.extend(airport_airline_cols)

for col in categorical_columns:
    if col in df_test.columns:
        # Convert to categorical if it has reasonable number of unique values
        unique_count = df_test[col].nunique()
        total_count = len(df_test)
        
        # Convert to categorical if less than 50% unique values (arbitrary threshold)
        if unique_count / total_count < 0.5:
            df_test[col] = df_test[col].astype('category')
            print(f"   ✓ {col}: {unique_count} categories")
        else:
            print(f"   → {col}: Too many unique values ({unique_count}), keeping as object")

# If you still have requestDate (maybe in original_df_test), use it:
if 'requestDate' in df_test.columns:
    df_test['requestDate'] = pd.to_datetime(df_test['requestDate'], errors='coerce')

    # Extract parts
    df_test['requestDate_hour'] = df_test['requestDate'].dt.hour
    df_test['requestDate_minute'] = df_test['requestDate'].dt.minute
    df_test['requestDate_dayofweek'] = df_test['requestDate'].dt.dayofweek
    df_test['requestDate_is_weekend'] = df_test['requestDate_dayofweek'].isin([5, 6]).astype(int)

    # Sin-Cos encoding for hour (cyclical feature)
    df_test['requestDate_hour_sin'] = np.sin(2 * np.pi * df_test['requestDate_hour'] / 24)
    df_test['requestDate_hour_cos'] = np.cos(2 * np.pi * df_test['requestDate_hour'] / 24)
else:
    print("❌ requestDate column not found in test data. Cannot engineer time features.")


Loaded 6897776 rows from 45231 sessions
Min options per session: 1
Max options per session: 7022
Starting missing value imputation...
Original missing values: 505902566
✓ Filled corporateTariffCode with -1 (converted to int64)
✓ Filled pricingInfo_isAccessTP with 0.0
Filled frequentFlyer with NONE
✓ Filled return flight timing fields with 'NONE'
✓ Filled return flight timing fields with 'NONE'
✓ Filled return flight timing fields with 'NONE'
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 object segment columns with 'NONE'
✓ Filled 32 float segment columns with 0.0
✓ Filled 48 ob

In [30]:
df_test_copy = df_test.copy()

In [ ]:
# # 3. Load trained model
# print("Loading model...")
# import joblib
# model = joblib.load("/kaggle/input/lgbm/other/default/1/lgb_ranker_model1.pkl")
# expected_features = model.booster_.feature_name()
# print(f"Model expects {len(expected_features)} features")

# # Replace 'df_test' with your actual test DataFrame
# current_features = set(df_test.columns) - {'Id', 'ranker_id'}  # Exclude ID columns
# print(f"Test data has {len(current_features)} features")

# # ✅ FIX: Convert both to sets before subtracting
# missing_in_test = set(expected_features) - current_features
# extra_in_test = current_features - set(expected_features)

# print(f"\n=== FEATURE MISMATCH ANALYSIS ===")
# print(f"Missing in test data ({len(missing_in_test)} features):")
# for feature in sorted(missing_in_test):
#     print(f"  - {feature}")

# print(f"\nExtra in test data ({len(extra_in_test)} features):")
# for feature in sorted(extra_in_test):
#     print(f"  - {feature}")


In [31]:
# ===========================
# Load Trained Model & Prepare Test Data
# ===========================
# Loads the previously saved LightGBM ranking model and prepares test data for inference.
#
# 🔹 Uses `joblib.load` to deserialize the trained model.
# 🔹 Extracts expected feature names from the model to ensure exact feature alignment.
# 🔹 Applies the same target encoder (`te`) to categorical columns as used during training.
# 🔹 Drops non-feature columns (like `ranker_id`) and reorders test features to match the model.
#
# ✅ This ensures the test input schema is **identical** to what the model was trained on.
# ⚠️ Mismatched feature order or missing columns will cause inference to fail — always align!

import joblib
# 3. Load trained model
print("Loading model...")
model = joblib.load("/kaggle/working/lgb_ranker_model3.pkl")
model_features = model.booster_.feature_name()
print(f"Model expects {len(model_features)} features: {model_features[:5]}...")

X_test = df_test_copy.drop(columns=['ranker_id'], errors='ignore')

X_test[cat_cols] = te.transform(X_test[cat_cols])

X_test = X_test[model_features]
#Ensure feature order matches
X_test_ordered = X_test[model_features]
print(f"Feature order aligned: {list(X_test_ordered.columns) == list(model_features)}")

Loading model...
Model expects 130 features: ['Id', 'bySelf', 'companyID', 'corporateTariffCode', 'frequentFlyer']...
Feature order aligned: True


In [32]:
# ===========================
# Prediction & Submission Generation
# ===========================
# Uses the trained LightGBM ranker model to generate predictions on the test dataset
# and prepares the final submission file in the required format.
#
# 🔹 `model.predict(X_test)`: Generates ranking scores for each flight option.
# 🔹 Constructs a submission DataFrame with:
#     - `Id`: Unique identifier for each flight option
#     - `ranker_id`: Session group identifier
#     - `selected`: Rank of the option within each session (1 = highest score)
#
# ⚠️ We use `groupby('ranker_id')` and `rank(..., ascending=False)` to ensure the highest score is ranked first.
# 🔹 Prediction scores (`pred`) are dropped from the final CSV — only `Id`, `ranker_id`, and `selected` are kept.
# 🔹 Submission row order is aligned with original test data for consistency.
#


# Predict using your trained LightGBM ranker model
print("Predicting....")
test_preds = model.predict(X_test)  # Make sure X_test matches training features
print("Successfully Predicted!")

Predicting....
Successfully Predicted!


In [31]:
X_test.shape
len(model_features)

130

In [33]:
# Assuming your test DataFrame is called test_df and has 'Id' and 'ranker_id' columns
submission_df = df_test_copy[['Id', 'ranker_id']].copy()
submission_df['pred'] = test_preds

# Rank within each session (ranker_id); higher score = better → rank 1 is best
submission_df['selected'] = submission_df.groupby('ranker_id')['pred'] \
    .rank(method='first', ascending=False).astype(int)


In [34]:
# Drop prediction scores (not needed in submission)
submission_df = submission_df.drop(columns=['pred'])

# Ensure row order is exactly like test_df
submission_df = submission_df.set_index('Id').loc[df_test['Id']].reset_index()

submission_df.head()
# Save as CSV
submission_df.to_csv('submission1.csv', index=False)
print("📦 Submission saved as submission.csv")


📦 Submission saved as submission.csv


# Current Leaderboard Status 🎯

We’ve achieved a **score of 0.31126** on the leaderboard so far — a solid baseline!

🚀 Next steps: Focus on **feature engineering** and model tuning to push this score higher.  
Stay tuned for updates as we iterate and refine the pipeline.  